In [0]:
import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers


In [0]:
def appendExt(fileName):
  return fileName+".jpg"

In [0]:
trainingDF = pd.read_csv("train.csv")
trainingDF["image_id"] = trainingDF["image_id"].apply(appendExt)

testingDF = pd.read_csv("test.csv")
testingDF["image_id"] = testingDF["image_id"].apply(appendExt)

In [0]:
imageSize = 100
batchSize = 32
epochs = 100

In [0]:
labels = ["healthy", "multiple_diseases", "rust", "scab"]
datagen = ImageDataGenerator(rescale = 1. / 255.,
                             rotation_range=45,
                             width_shift_range=.15,
                             height_shift_range=.15,
                             horizontal_flip=True,
                             zoom_range=0.5
)
validationDataGen = ImageDataGenerator(rescale = 1. / 255.)
testDataGen = ImageDataGenerator(rescale = 1. / 255.)

In [0]:
trainGenerator = datagen.flow_from_dataframe(
    dataframe = trainingDF[:1455],
    directory = 'images',
    x_col = "image_id",
    y_col = labels,
    batch_size = batchSize,
    seed = 42,
    shuffle = True,
    class_mode = "raw",
    target_size=(imageSize,imageSize)
)

validationGenerator = validationDataGen.flow_from_dataframe(
    dataframe = trainingDF[1456:],
    directory = 'images',
    x_col = "image_id",
    y_col = labels,
    batch_size = batchSize,
    seed = 42,
    shuffle = True,
    class_mode = "raw",
    target_size=(imageSize,imageSize)
)

testGenerator = testDataGen.flow_from_dataframe(
    dataframe = testingDF,
    directory = 'images',
    x_col = "image_id",
    batch_size = 1,
    seed = 42,
    shuffle = False,
    class_mode = None,
    target_size=(imageSize,imageSize)
)

Found 1455 validated image filenames.
Found 365 validated image filenames.
Found 1821 validated image filenames.


In [0]:
model = Sequential([
  Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(imageSize, imageSize, 3)),
  Conv2D(32, (3,3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Conv2D(64, (3,3), padding='same', activation='relu'),
  Conv2D(64, (3,3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Flatten(),
  Dense(512, activation='relu'),
  Dropout(0.5),
  Dense(4, activation='softmax')
])

model.compile(optimizers.Adam(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 50, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 25, 25, 64)       

In [77]:
STEP_SIZE_TRAIN = trainGenerator.n//trainGenerator.batch_size
STEP_SIZE_VALID = validationGenerator.n//validationGenerator.batch_size
model.fit_generator(generator=trainGenerator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validationGenerator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs
)

Epoch 1/100
45/45 [==============================] - 83s 2s/step - loss: 0.4333 - accuracy: 0.8609 - val_loss: 1.0172 - val_accuracy: 0.8210
Epoch 2/100
45/45 [==============================] - 82s 2s/step - loss: 0.4294 - accuracy: 0.8630 - val_loss: 0.3351 - val_accuracy: 0.8288
Epoch 3/100
45/45 [==============================] - 83s 2s/step - loss: 0.4071 - accuracy: 0.8658 - val_loss: 0.7283 - val_accuracy: 0.8408
Epoch 4/100
45/45 [==============================] - 82s 2s/step - loss: 0.4320 - accuracy: 0.8623 - val_loss: 0.4838 - val_accuracy: 0.8018
Epoch 5/100
45/45 [==============================] - 83s 2s/step - loss: 0.4565 - accuracy: 0.8458 - val_loss: 0.5754 - val_accuracy: 0.8078
Epoch 6/100
45/45 [==============================] - 84s 2s/step - loss: 0.4170 - accuracy: 0.8663 - val_loss: 0.5394 - val_accuracy: 0.8619
Epoch 7/100
45/45 [==============================] - 81s 2s/step - loss: 0.4253 - accuracy: 0.8665 - val_loss: 0.6403 - val_accuracy: 0.8408
Epoch 8/100
4

In [78]:
STEP_SIZE_TEST = testGenerator.n//testGenerator.batch_size
testGenerator.reset()
pred=model.predict_generator(testGenerator,
steps=STEP_SIZE_TEST,
verbose=1)

1821/1821 [==============================] - 43s 24ms/step


In [0]:
pred_bool = (pred > 0.5)

In [0]:
predictions = pred_bool.astype(int)
results = pd.DataFrame()
results['image_id']= testingDF['image_id']
results['healthy']=predictions[:,0]
results['multiple_diseases']=predictions[:,1]
results['rust']=predictions[:,2]
results['scab']=predictions[:,3]
results.to_csv("results_7.csv", index=False)